# Using siibra-python to characterize motor and language areas in the Julich-Brain cytoarchitectonic atlas 


### 1. Import the package 

We start by importing the `siibra` package.

In [ ]:
import siibra
from packaging.version import Version
assert Version(siibra.__version__) >= Version('1.0.1-alpha.2')

import matplotlib.pyplot as plt
%matplotlib notebook

### 2. Select two regions from Julich-Brain

In [ ]:
# access Julich-Brain in version 2.9
julichbrain = siibra.parcellations.get('julich')

# Get two region objects from Julich-Brain
regions = [
    julichbrain.get_region('4p left'),
    julichbrain.get_region('44 left')
]

In [ ]:
julich_mni_pmaps = julichbrain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC, 
    maptype=siibra.MapType.STATISTICAL
)

In [ ]:
# fetch and display their probability maps
from nilearn import plotting
for region in regions:
    pmap = julich_mni_pmaps.fetch(region)
    plotting.plot_stat_map(pmap, title=region.name)

### 4. Access GABAB receptor distributions

In [ ]:
receptor = 'GABAB'

In [ ]:
fig, axs = plt.subplots(1, len(regions), figsize=(8, 3), sharex=True, sharey=True)

for i, region in enumerate(regions):
    
    # retrieve receptor profiles
    features = siibra.features.get(
        region, siibra.features.molecular.ReceptorDensityProfile
    )
    
    # get one with GABAB type and plot it
    features[0].get_element('GABAB').plot(ax=axs[i])

plt.tight_layout()

### 5. Run a differential gene expression analysis for the two regions

In [ ]:
# setup and run the analysis
genes = ['GABBR1', 'GABBR2']

fig, axs = plt.subplots(1, len(regions), figsize=(8, 3), sharex=True, sharey=True)

geneexps = {}
for i, region in enumerate(regions):
    # retrieve receptor profiles
    geneexps[region] = siibra.features.get(region, "GeneExpression", gene=genes)[0]
    # get one with GABAB type and plot it
    geneexps[region].plot(ax=axs[i])

plt.tight_layout()

In [ ]:
# plot the extracted sample positions of the microarray data
from nilearn import plotting
for region in regions:
    pmap = region.get_regional_map(
        siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
        siibra.MapType.STATISTICAL
    )
    display = plotting.plot_glass_brain(pmap.fetch(), cmap="viridis", title=region.name)
    display.add_markers(geneexps[region].anchor.location.coordinates, marker_size=5)